In [2]:
from IPython.display import Markdown, display

import pandas as pd
import numpy as np
import pickle


# plots
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import missingno as msno
import plotly.figure_factory as ff
import plotly.graph_objects as go


from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import TargetEncoder, PolynomialFeatures, StandardScaler
import category_encoders as ce
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline


from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import optuna

from scipy.stats import gaussian_kde, boxcox, skew
import logging
from tqdm.notebook import trange
import itertools

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

pd.set_option('display.max_colwidth', None)
np.random.seed(42)
rng = np.random.default_rng(42)
from pathlib import Path
data_path = Path('./data')

In [3]:
with open('initial_eda_df.pkl', 'rb') as file:
    initial_eda_df = pickle.load(file)

In [4]:
initial_eda_df

,market_id,created_at,store_id,store_primary_category,order_protocol,total_items,total_onshift_dashers,estimated_store_to_consumer_driving_duration,actual_duration,subtotal_bc,min_item_price_bc,max_price_bc,order_duration_bc,day,hour,dasher_ratio,net_dashers_orders
0,1.0,2015-02-06 22:24:17,1845,2868.516769,1.0,4.0,33.0,861.0,3779.0,7.670465,12.989654,18.370412,2.002660,4,22,0.424242,12.0
1,2.0,2015-02-10 21:49:25,5477,2673.598786,2.0,1.0,1.0,690.0,4024.0,7.141866,16.717004,19.027647,2.002660,1,21,2.000000,-1.0
2,3.0,2015-01-22 20:39:28,5477,2984.153316,1.0,1.0,1.0,690.0,1781.0,7.141866,18.116326,20.755206,2.002660,3,20,0.000000,1.0
3,3.0,2015-02-03 21:21:45,5477,2984.153316,1.0,6.0,1.0,289.0,3075.0,8.283814,13.264879,20.440274,2.002660,1,21,1.000000,-1.0
4,3.0,2015-02-15 02:40:36,5477,2984.153316,1.0,3.0,6.0,650.0,2390.0,7.781316,15.672436,19.767861,2.002660,6,2,1.000000,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,1.0,2015-02-17 00:19:41,2956,2657.739372,4.0,3.0,17.0,331.0,3907.0,6.861162,11.314232,15.188312,1.965378,1,0,1.000000,-6.0
197424,1.0,2015-02-13 00:01:59,2956,2657.739372,4.0,6.0,12.0,915.0,3383.0,7.551765,11.856762,16.313275,1.965378,4,0,0.916667,-2.0
197425,1.0,2015-01-24 04:46:08,2956,2657.739372,4.0,5.0,39.0,795.0,3008.0,7.111227,10.855721,13.092747,1.965378,5,4,1.051282,-1.0
197426,1.0,2015-02-01 18:18:15,3630,2685.420236,1.0,1.0,7.0,384.0,3907.0,6.710704,12.842257,14.327525,2.002660,6,18,1.000000,-5.0


In [5]:
X = initial_eda_df.drop(columns=['actual_duration', 'created_at'])
y = initial_eda_df['actual_duration']

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 193832 entries, 0 to 197427
Data columns (total 15 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     193832 non-null  float64
 1   store_id                                      193832 non-null  int64  
 2   store_primary_category                        193832 non-null  object 
 3   order_protocol                                193832 non-null  float64
 4   total_items                                   193832 non-null  float64
 5   total_onshift_dashers                         193832 non-null  float64
 6   estimated_store_to_consumer_driving_duration  193832 non-null  float64
 7   subtotal_bc                                   193832 non-null  float64
 8   min_item_price_bc                             193832 non-null  float64
 9   max_price_bc                                  193832 

In [7]:
corr_matrix = X.corr()

mask = (corr_matrix.abs() > 0.8) & (corr_matrix != 1)
columns_corr = mask.any(axis=0)

corr_m = corr_matrix[corr_matrix.abs() > 0.7]
corr_pairs = corr_m.stack().reset_index()
corr_pairs.columns = ['column1', 'column2', 'correlation']
corr_pairs.loc[corr_pairs['column1'] == corr_pairs['column2']]
corr_pairs = corr_pairs.drop(corr_pairs.loc[corr_pairs['column1'] == corr_pairs['column2']].index)

corr_pairs.drop_duplicates(subset='correlation')

,column1,column2,correlation


### initial_eda

#### linear Regression

In [8]:
lr = LinearRegression()
lr.fit(xtrain, ytrain)
lr_preds = lr.predict(xtest)

lr_r2 = r2_score(lr_preds, ytest)
lr_rmse = np.sqrt(mean_squared_error(lr_preds, ytest))

print(f'linear regression r^2: {lr_r2}')
print(f'linear regression rmse: {lr_rmse}')

linear regression r^2: -13.307713688873656
linear regression rmse: 1982.4093233051276


#### polynomial regression

In [9]:
pipeline = make_pipeline(PolynomialFeatures(), StandardScaler(), Ridge())

##### poly1

param_grid1 = {
    'polynomialfeatures__degree': [1, 2, 3],
    'ridge__alpha': [10, 100, 1000] 
}

In [10]:
param_grid1 = {
    'polynomialfeatures__degree': [1,2,3],
    'ridge__alpha': [10, 100, 1000] 
}

grid_search1 = GridSearchCV(pipeline, param_grid1, cv=5, scoring='neg_mean_squared_error', n_jobs=1)
grid_search1.fit(xtrain, ytrain)

poly1_best_model = grid_search1.best_estimator_
poly1_best_param = grid_search1.best_params_
poly1_preds = poly1_best_model.predict(xtest)

poly1_r2 = r2_score(poly1_preds, ytest)
poly1_rmse = np.sqrt(mean_squared_error(poly1_preds, ytest))

print(f'polynomial regression r^2: {poly1_r2}')
print(f'polynomial regression rmse: {poly1_rmse}')
print(f'best parameters: {poly1_best_param}')

polynomial regression r^2: -9.70862638252411
polynomial regression rmse: 1963.8581062571138
best parameters: {'polynomialfeatures__degree': 3, 'ridge__alpha': 10}


#### random forest regression

In [11]:
rf = RandomForestRegressor(random_state=42)

In [12]:

rf_param = {
    'n_estimators': [20, 30, 50],
    'max_depth': [20, 30, 50],
    'min_samples_leaf': [5,7,10]
}

rf_search = RandomizedSearchCV(RandomForestRegressor(random_state=42), rf_param, n_iter=10, cv=5, n_jobs=1, random_state=42)
rf_search.fit(xtrain, ytrain)

rf_best_parm = rf_search.best_params_
rf_best_model = rf_search.best_estimator_

rf_best_preds = rf_best_model.predict(xtest)

rf_r2 = r2_score(rf_best_preds, ytest)
rf_rmse = np.sqrt(mean_squared_error(rf_best_preds, ytest))

print(f'polynomial regression r^2: {rf_r2}')
print(f'polynomial regression rmse: {rf_rmse}')

polynomial regression r^2: -9.054502307047215
polynomial regression rmse: 1955.6035057372533


In [13]:
rf_best_parm

{'n_estimators': 50, 'min_samples_leaf': 10, 'max_depth': 30}

In [14]:
rf = RandomForestRegressor(random_state=42)
rf_param = {
    'n_estimators': [20, 30, 50],
    'max_depth': [20, 30, 50],
    'min_samples_leaf': [20,25,30]
}

rf_search = RandomizedSearchCV(RandomForestRegressor(random_state=42), rf_param, n_iter=10, cv=5, n_jobs=1, random_state=42)
rf_search.fit(xtrain, ytrain)

rf_best_parm = rf_search.best_params_
rf_best_model = rf_search.best_estimator_

rf_best_preds = rf_best_model.predict(xtest)

rf_r2 = r2_score(rf_best_preds, ytest)
rf_rmse = np.sqrt(mean_squared_error(rf_best_preds, ytest))

print(f'polynomial regression r^2: {rf_r2}')
print(f'polynomial regression rmse: {rf_rmse}')

polynomial regression r^2: -9.951405101506019
polynomial regression rmse: 1956.7849298068531


In [15]:
rf_best_parm

{'n_estimators': 50, 'min_samples_leaf': 30, 'max_depth': 30}

### subsets

In [16]:
def fit_linear_reg(X_train, y_train, X_test, y_test):
  model = LinearRegression()
  model.fit(X_train, y_train)

  n = len(y_test)
  p = X_test.shape[1] + 1

  ypred = model.predict(X_test)
  sse = sum((y_test-ypred)**2)
  mse = mean_squared_error(y_test, ypred)
  mae = mean_absolute_error(y_test, ypred)
  r2 = model.score(X_test, y_test)
  adj_r2 = 1 - ((n-1)/(n-p))*(1-r2)

  return mse, mae, r2, adj_r2

In [29]:
def best_subset(X, y):

  mse_list, mae_list, r2_list, adj_r2_list, features = [], [], [], [], []
  xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

  for k in trange(1, X.shape[1]+1, desc = 'Loop...'):              # p choose k

   for combo in itertools.combinations(xtrain.columns, k):
      tmp_results = fit_linear_reg(xtrain[list(combo)], ytrain, xtest[list(combo)], ytest)
      mse_list.append(tmp_results[0])
      mae_list.append(tmp_results[1])
      r2_list.append(tmp_results[2])
      adj_r2_list.append(tmp_results[3])
      features.append(list(combo))

  return mse_list, mae_list, r2_list, adj_r2_list, features

In [ ]:
mse_list, mae_list, r2_list, adj_r2_list, features = best_subset(X, y)